# Versions Python et Libraries

In [1]:
!python --version

Python 3.12.3


In [2]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
# import shutil

In [5]:
# Modélisation CNN avec tensorflow
import tensorflow as tf
import tensorflow.keras.layers as tfl

# Gestion des images : lecture, transformations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

# Gestion de l'architecture du réseau
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Conv2D, Activation, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Resizing, Rescaling, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

# Architecture de modèles de réseaux pré-entrainés (fonctionnalité de Transfer Learning)
from tensorflow.keras.applications import MobileNetV2, VGG16

# Algorithme d'optimisation
from tensorflow.keras.optimizers import RMSprop, Adam

# Sauvegarde, arret
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

# Open CV
import cv2

# Performances des modeles
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.utils.multiclass import unique_labels

# Constantes

In [7]:
# chemin des 2 dossiers d'images par classe à prédire pour l'apprentissage et l'évaluation des performances et l'inférence
SRC_PATH_TRAIN = "./dataset_footprint/train/"
SRC_PATH_TEST = "./dataset_footprint/test/"

# Liste des catégories (classes à prévoir)
LST_LABELS = os.listdir(SRC_PATH_TRAIN)

# Parametres pour la generation d'images
SEED_VALUE = 42
VALID_SIZE = 0.2

In [8]:
IMG_SIZE = 160  # Taille de l'image IMG_SIZExIMG_SIZE (on augmente la resolution de 100 a 160)
BATCH_SIZE = 10  # nb de données à passer pour un A/R dans le réseau (total de 77 images x 3 classes)
NB_EPOCHS = 30  # Nb de passes

In [9]:
# Définition comme label les noms des sous-dossiers de travail
labels = os.listdir(SRC_PATH_TRAIN)
LST_DIR_LABELS = labels
LST_DIR_LABELS

['Castor',
 'Chat',
 'Chien',
 'Coyote',
 'Ecureuil',
 'Lapin',
 'Loup',
 'Lynx',
 'Ours',
 'Puma',
 'Rat',
 'RatonLaveur',
 'Renard']

In [19]:
# Sauvegarde du modèle optimal (nom et sous-dossier)
CKPT_NO, MDL_NAME = 'ckpt_footprints_1', '3footprints_CNN'
CKPT_DIR = './'+ CKPT_NO
PATH_BEST_MDL = CKPT_DIR + '/' + MDL_NAME

# Fonctions Locales

## Graphs

In [11]:
def plot_learning_curve(history):
    """
    Fonction de tracé de la courbe d'ajustement d'un modèle
    Arguments:
        history : sequence de recueil des métriques d'évaluation d'un modèle lors de la phase d'apprentissage
        les métriques sont ici prédéfinies : 'accuracy','val_accuracy','loss','val_loss'
    Returns:
        2 figures Matplotlib superposées des métriques 'accuracy' et 'loss' sur les datasets TRAIN et VALIDATION
    """
    # Définition des séquences de sauvegarde des performances en TRAIN (accuracy et loss) et VALIDATION (val_)
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    # Liste des itérations de calcul : epochs
    lst_epochs = range(len(accuracy))
    
    # Tracé en 2 figures
    plt.figure(figsize=(20,6))
    plt.plot(lst_epochs, accuracy, "b", label="accuracy [TRAIN]")
    plt.plot(lst_epochs, val_accuracy, "r", label="accuracy [VALIDATION]")
    plt.title("Exactitude du modèle")
    plt.legend()
    plt.show()

    plt.figure(figsize=(20,6))
    plt.plot(lst_epochs, loss, "b", label="loss [TRAIN]")
    plt.plot(lst_epochs, val_loss, "r", label="loss [VALIDATION]")
    plt.title("Courbe de perte du modèle")    
    plt.legend()
    plt.show()

In [12]:
# WARNING THIS FUNCTION IS BASICALLY UNUSED

# create a confusion matrix to visually represent incorrectly classified images
def plot_confusion_matrix(y_true, y_pred, classes, out_path=""):
    cm = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cm, index=[i for i in classes], columns=[i for i in classes])
    plt.figure(figsize=(3,3))
    ax = sns.heatmap(df_cm, annot=True, square=True, fmt="d", linewidths=.2, cbar_kws={"shrink": 0.8})
    if out_path:
        plt.savefig(out_path + "/confusion_matrix.png")  # as in the plot_model_history, the matrix is saved in a file called "model_name_confusion_matrix.png"
    return ax

# Définition de l'architecture du réseau de neurones convolutifs (CNN)

In [22]:
def prepare_model():
    # Initialisation du réseau
    model = Sequential()
    model.add(Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
    
    # Blocs de Convolution
    # model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3), padding='same'))
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
   
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Couches de classification
    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    
    #model.add(Dense(16, activation='relu'))
    #model.add(Dropout(0.3))
    
    # Couche de sortie
    model.add(Dense(len(LST_LABELS), activation='softmax'))
    
    return model

In [14]:
# Fonction d'inférence avec le modele model de l'image n° numero du dossier src_path de la classe categorie
def f_footprint_predict(categorie, numero, model, src_path=SRC_PATH_TEST, lst_labels=LST_LABELS):
    # Image à classifier dans la catégorie courante
    id_image = categorie + '_' + numero + '.jpg'
    lb_image = src_path + '/' + categorie + '/' + id_image
    print('Image :', lb_image)

    # Lecture et normalisation de l'image
    img = load_img(lb_image, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img, dtype=np.uint8)
    img_array = np.array(img_array)/255.0
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    # Affichage
    img = cv2.imread(lb_image)
    plt.figure(figsize=(4,4))
    print(plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
    plt.show()

    # Prédiction et évaluation
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    # Affichage des résultats
    print("Cette image appartient probablement à la classe {} avec un niveau de confiance à {:.2f}."
        .format(lst_labels[np.argmax(score)], 100 * np.max(score)))

    print(tf.nn.softmax(predictions).numpy())
    return predictions, score

In [15]:
def print_layer_trainable(model):
    ''' Statut et nom des couches d'un modèle
    '''
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

# Preparation des Données

In [16]:
# Objet générateur de données de type images 
# Séquence de transformation à appliquer à la volée : normalisation (taille d'image), rotation, zoom, ...
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2, 
        horizontal_flip=True,    
        rotation_range=20,
        width_shift_range=0.2,  # 0.05
        height_shift_range=0.2,
        #fill_mode="nearest",
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
# générateur qui définit à la volée des données à partir du jeu de données sources
train_generator = train_datagen.flow_from_directory(
    directory=SRC_PATH_TRAIN,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode='sparse',  # binaire <=> vecteurs de proba, sparse <=> index de la classe, categorical
    #classes=LST_LABELS,
    subset='training',
    shuffle=True,
    seed=SEED_VALUE
)

labels = (train_generator.class_indices)
print(labels,'\n')

valid_generator = train_datagen.flow_from_directory(
    directory=SRC_PATH_TRAIN,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode='sparse',   # "sparse",
    #classes=LST_LABELS,
    subset='validation',
    shuffle=True,
    seed=SEED_VALUE
)

test_generator = test_datagen.flow_from_directory(
    directory=SRC_PATH_TEST,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=1,
    class_mode='sparse',   #None,
    shuffle=False,
    seed=SEED_VALUE  # pas utile ici
)

Found 176 images belonging to 13 classes.
{'Castor': 0, 'Chat': 1, 'Chien': 2, 'Coyote': 3, 'Ecureuil': 4, 'Lapin': 5, 'Loup': 6, 'Lynx': 7, 'Ours': 8, 'Puma': 9, 'Rat': 10, 'RatonLaveur': 11, 'Renard': 12} 

Found 37 images belonging to 13 classes.
Found 52 images belonging to 13 classes.


# Construction du modèle et apprentissage des données

In [24]:
# Instanciation du modèle CNN
model = prepare_model()

# Compilation du modèle
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=['accuracy'])

# Apprentissage sur le dataset TRAIN avec confrontation des performances avec le sous-ensemble de VALidation (20%)
history = model.fit(train_generator, 
          validation_data=valid_generator,
          verbose=1,
          steps_per_epoch=train_generator.n//train_generator.batch_size,
          validation_steps=valid_generator.n//valid_generator.batch_size,
          epochs=NB_EPOCHS,
          batch_size=BATCH_SIZE,
          callbacks=[early_stopping]
        #   callbacks=keras_callbacks
)

Epoch 1/30


c:\Users\louis\Python Projects\ia-wildlens\epsiEnv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - accuracy: 0.0660 - loss: 2.6124 - val_accuracy: 0.1000 - val_loss: 2.5433
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 2.5594 - val_accuracy: 0.2857 - val_loss: 2.5445
Epoch 3/30


C:\Users\louis\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 229ms/step - accuracy: 0.1207 - loss: 2.5528 - val_accuracy: 0.1000 - val_loss: 2.5202
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3000 - loss: 2.5046 - val_accuracy: 0.2857 - val_loss: 2.5048
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 225ms/step - accuracy: 0.1308 - loss: 2.5399 - val_accuracy: 0.1333 - val_loss: 2.5071
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1000 - loss: 2.5316 - val_accuracy: 0.0000e+00 - val_loss: 2.5468
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 230ms/step - accuracy: 0.1445 - loss: 2.5239 - val_accuracy: 0.1333 - val_loss: 2.5423
Epoch 7: early stopping


In [20]:
# Prepare a directory to store all the checkpoints.
if not os.path.exists(CKPT_DIR):
    os.makedirs(CKPT_DIR)

# Configuration de keras_callbacks pour la sauvegarde du modèle optimal

# Configuration de l'arrêt anticipé par monitoring de la loss ou de l'accuracy (à expérimenter)
#early_stopping  = EarlyStopping(monitor='val_loss', patience=5, mode='auto', min_delta=0, verbose=1)
early_stopping  = EarlyStopping(monitor='val_accuracy', patience=5, mode='auto', min_delta=0, verbose=1)

keras_callbacks = [ModelCheckpoint(filepath=PATH_BEST_MDL, 
                                   monitor='val_accuracy', 
                                   save_best_only=True, 
                                   mode='auto'),
                   early_stopping]

ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=./ckpt_footprints_1/3footprints_CNN